![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Adverse Drug Events Detection using Named Entity Recognition, Classification and Assertion Status Models

`ADE NER`: Extracts ADE and DRUG entities from clinical texts.

`ADE Classifier`: CLassify if a sentence is ADE-related (`True`) or not (`False`)

We use several datasets to train these models:

- Twitter dataset, which is used in paper "`Deep learning for pharmacovigilance: recurrent neural network architectures for labeling adverse drug reactions in Twitter posts`" (https://pubmed.ncbi.nlm.nih.gov/28339747/)
- ADE-Corpus-V2, which is used in paper "`An Attentive Sequence Model for Adverse Drug Event Extraction from Biomedical Text`" (https://arxiv.org/abs/1801.00625) and availe online: https://sites.google.com/site/adecorpus/home/document.
- CADEC dataset, which is sued in paper `Cadec: A corpus of adverse drug event annotations` (https://pubmed.ncbi.nlm.nih.gov/25817970)

In [0]:
import os
import json
import string
import numpy as np
import pandas as pd


import sparknlp
import sparknlp_jsl
from sparknlp.base import *
from sparknlp.util import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
#from sparknlp.pretrained import ResourceDownloader

from pyspark.sql import functions as F
from pyspark.ml import Pipeline, PipelineModel

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', False)


print('sparknlp_jsl.version : ',sparknlp_jsl.version())

spark

sparknlp_jsl.version : 3.0.1
Out[1]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.0.1 
 Master 
 spark://10.164.237.178:7077 
 AppName 
 Databricks Shell

## ADE Classifier Pipeline (with a pretrained model)

`True` : The sentence is talking about a possible ADE

`False` : The sentences doesn't have any information about an ADE.

In [0]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

bert_embeddings = BertEmbeddings.pretrained("biobert_pubmed_base_cased")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")\
    .setMaxSentenceLength(512)

embeddingsSentence = SentenceEmbeddings() \
    .setInputCols(["sentence", "embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")\
    .setStorageRef('biobert_pubmed_base_cased')

classsifierdl = ClassifierDLModel.pretrained("classifierdl_ade_biobert", "en", "clinical/models")\
    .setInputCols(["sentence", "sentence_embeddings"]) \
    .setOutputCol("class")

ade_clf_pipeline = Pipeline(
    stages=[documentAssembler, 
            tokenizer,
            bert_embeddings,
            embeddingsSentence,
            classsifierdl])


empty_data = spark.createDataFrame([[""]]).toDF("text")
ade_clf_model = ade_clf_pipeline.fit(empty_data)

ade_lp_pipeline = LightPipeline(ade_clf_model)

biobert_pubmed_base_cased download started this may take some time.
Approximate size to download 386.4 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]
classifierdl_ade_biobert download started this may take some time.
Approximate size to download 21.8 MB
[ | ][ / ][ — ][OK!]

In [0]:
text = "I feel a bit drowsy & have a little blurred vision after taking an insulin"

ade_lp_pipeline.annotate(text)['class'][0]

Out[3]: 'True'

In [0]:
text="I just took an Advil and have no gastric problems so far."

ade_lp_pipeline.annotate(text)['class'][0]

Out[4]: 'False'

As you see `gastric problems` is not detected as `ADE` as it is in a negative context. So, classifier did a good job detecting that.

In [0]:
text="I just took a Metformin and started to feel dizzy."

ade_lp_pipeline.annotate(text)['class'][0]

Out[5]: 'True'

In [0]:
t='''
Always tired, and possible blood clots. I was on Voltaren for about 4 years and all of the sudden had a minor stroke and had blood clots that traveled to my eye. I had every test in the book done at the hospital, and they couldnt find anything. I was completley healthy! I am thinking it was from the voltaren. I have been off of the drug for 8 months now, and have never felt better. I started eating healthy and working out and that has help alot. I can now sleep all thru the night. I wont take this again. If I have the back pain, I will pop a tylonol instead.
'''

ade_lp_pipeline.annotate(t)['class'][0]


Out[6]: 'True'

In [0]:
texts = ["I feel a bit drowsy & have a little blurred vision, after taking a pill.",
"I've been on Arthrotec 50 for over 10 years on and off, only taking it when I needed it.",
"Due to my arthritis getting progressively worse, to the point where I am in tears with the agony, gp's started me on 75 twice a day and I have to take it every day for the next month to see how I get on, here goes.",
"So far its been very good, pains almost gone, but I feel a bit weird, didn't have that when on 50."]

for text in texts:

  result = ade_lp_pipeline.annotate(text)

  print (result['class'][0])


True
False
True
False

### ADE Classifier trained with conversational (short) sentences

This model is trained on short, conversational sentences related to ADE and is supposed to do better on the text that is short and used in a daily context.

In [0]:
conv_classsifierdl = ClassifierDLModel.pretrained("classifierdl_ade_conversational_biobert", "en", "clinical/models")\
            .setInputCols(["sentence", "sentence_embeddings"]) \
            .setOutputCol("class")

conv_ade_clf_pipeline = Pipeline(
    stages=[documentAssembler, 
            tokenizer,
            bert_embeddings,
            embeddingsSentence,
            conv_classsifierdl])

empty_data = spark.createDataFrame([[""]]).toDF("text")
conv_ade_clf_model = conv_ade_clf_pipeline.fit(empty_data)

conv_ade_lp_pipeline = LightPipeline(conv_ade_clf_model)

classifierdl_ade_conversational_biobert download started this may take some time.
Approximate size to download 21.8 MB
[ | ][ / ][ — ][OK!]

In [0]:
text = "after taking a pill, he denies any pain"

conv_ade_lp_pipeline.annotate(text)['class'][0]

Out[9]: 'False'

## ADE NER

Extracts `ADE` and `DRUG` entities from text.

In [0]:
documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

sentenceDetector = SentenceDetector()\
  .setInputCols(["document"])\
  .setOutputCol("sentence")

tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
  .setInputCols(["sentence", "token"])\
  .setOutputCol("embeddings")

ade_ner = MedicalNerModel.pretrained("ner_ade_clinical", "en", "clinical/models") \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("ner")

ner_converter = NerConverter() \
  .setInputCols(["sentence", "token", "ner"]) \
  .setOutputCol("ner_chunk")

ner_pipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    ade_ner,
    ner_converter])

empty_data = spark.createDataFrame([[""]]).toDF("text")

ade_ner_model = ner_pipeline.fit(empty_data)

ade_ner_lp = LightPipeline(ade_ner_model)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]
ner_ade_clinical download started this may take some time.
Approximate size to download 13.9 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]

In [0]:
light_result = ade_ner_lp.fullAnnotate("I feel a bit drowsy & have a little blurred vision, so far no gastric problems. I have been on Arthrotec 50 for over 10 years on and off, only taking it when I needed it. Due to my arthritis getting progressively worse, to the point where I am in tears with the agony, gp's started me on 75 twice a day and I have to take it every day for the next month to see how I get on, here goes. So far its been very good, pains almost gone, but I feel a bit weird, didn't have that when on 50.")

chunks = []
entities = []
begin =[]
end = []

for n in light_result[0]['ner_chunk']:

    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 

import pandas as pd

df = pd.DataFrame({'chunks':chunks, 'entities':entities,
                    'begin': begin, 'end': end})

df

Out[11]:

,chunks,entities,begin,end
0,bit drowsy,ADE,9,18
1,little blurred vision,ADE,29,49
2,Arthrotec,DRUG,95,103


As you see `gastric problems` is not detected as `ADE` as it is in a negative context. So, NER did a good job ignoring that.

#### ADE NER with Bert embeddings

In [0]:
documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

sentenceDetector = SentenceDetector()\
  .setInputCols(["document"])\
  .setOutputCol("sentence")

tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("token")

bert_embeddings = BertEmbeddings.pretrained("biobert_pubmed_base_cased")\
  .setInputCols(["sentence", "token"])\
  .setOutputCol("embeddings")
  
ade_ner_bert = MedicalNerModel.pretrained("ner_ade_biobert", "en", "clinical/models") \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("ner")

ner_converter = NerConverter() \
  .setInputCols(["sentence", "token", "ner"]) \
  .setOutputCol("ner_chunk")

ner_pipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    bert_embeddings,
    ade_ner_bert,
    ner_converter])

empty_data = spark.createDataFrame([[""]]).toDF("text")

ade_ner_model_bert = ner_pipeline.fit(empty_data)

ade_ner_lp_bert = LightPipeline(ade_ner_model_bert)

biobert_pubmed_base_cased download started this may take some time.
Approximate size to download 386.4 MB
[ | ][OK!]
ner_ade_biobert download started this may take some time.
Approximate size to download 15.3 MB
[ | ][ / ][OK!]

In [0]:
light_result = ade_ner_lp_bert.fullAnnotate("I feel a bit drowsy & have a little blurred vision, so far no gastric problems. I have been on Arthrotec 50 for over 10 years on and off, only taking it when I needed it. Due to my arthritis getting progressively worse, to the point where I am in tears with the agony, gp's started me on 75 twice a day and I have to take it every day for the next month to see how I get on, here goes. So far its been very good, pains almost gone, but I feel a bit weird, didn't have that when on 50.")

chunks = []
entities = []
begin =[]
end = []

for n in light_result[0]['ner_chunk']:

    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 

import pandas as pd

df = pd.DataFrame({'chunks':chunks, 'entities':entities,
                    'begin': begin, 'end': end})

df

Out[13]:

,chunks,entities,begin,end
0,drowsy,ADE,13,18
1,blurred vision,ADE,36,49
2,gastric problems,ADE,62,77
3,Arthrotec,DRUG,95,103
4,gp's,DRUG,269,272
5,pains,ADE,413,417


Looks like Bert version of NER returns more entities than clinical embeddings version.

## NER and Classifier combined with AssertionDL Model

In [0]:
assertion_ner_converter = NerConverter() \
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ass_ner_chunk")\
    .setWhiteList(['ADE'])

biobert_assertion = AssertionDLModel.pretrained("assertion_dl_biobert", "en", "clinical/models") \
    .setInputCols(["sentence", "ass_ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")

assertion_ner_pipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    bert_embeddings,
    ade_ner_bert,
    ner_converter,
    assertion_ner_converter,
    biobert_assertion])

empty_data = spark.createDataFrame([[""]]).toDF("text")

ade_ass_ner_model_bert = assertion_ner_pipeline.fit(empty_data)

ade_ass_ner_model_lp_bert = LightPipeline(ade_ass_ner_model_bert)

assertion_dl_biobert download started this may take some time.
Approximate size to download 3 MB
[ | ][ / ][OK!]

In [0]:
import pandas as pd
text = "I feel a bit drowsy & have a little blurred vision, so far no gastric problems. I have been on Arthrotec 50 for over 10 years on and off, only taking it when I needed it. Due to my arthritis getting progressively worse, to the point where I am in tears with the agony, gp's started me on 75 twice a day and I have to take it every day for the next month to see how I get on, here goes. So far its been very good, pains almost gone, but I feel a bit weird, didn't have that when on 50."

print (text)

light_result = ade_ass_ner_model_lp_bert.fullAnnotate(text)[0]

chunks=[]
entities=[]
status=[]

for n,m in zip(light_result['ass_ner_chunk'],light_result['assertion']):
    
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    status.append(m.result)
        
df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status})

df

I feel a bit drowsy & have a little blurred vision, so far no gastric problems. I have been on Arthrotec 50 for over 10 years on and off, only taking it when I needed it. Due to my arthritis getting progressively worse, to the point where I am in tears with the agony, gp's started me on 75 twice a day and I have to take it every day for the next month to see how I get on, here goes. So far its been very good, pains almost gone, but I feel a bit weird, didn't have that when on 50.
Out[15]:

,chunks,entities,assertion
0,drowsy,ADE,present
1,blurred vision,ADE,present
2,gastric problems,ADE,absent
3,pains,ADE,present


Looks great ! `gastric problems` is detected as `ADE` and `absent`

## ADE models applied to Spark Dataframes

In [0]:
import pyspark.sql.functions as F

! wget -q	https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/sample_ADE_dataset.csv

dbutils.fs.cp("file:/databricks/driver/sample_ADE_dataset.csv", "dbfs:/")

Out[16]: True

In [0]:
%sh cd /databricks/driver && ls -la && pwd

total 40
drwxr-xr-x 1 root root 4096 May 3 14:01 .
drwxr-xr-x 1 root root 4096 May 3 13:25 ..
drwxr-xr-x 2 root root 4096 Jan 1 1970 conf
drwxr-xr-x 3 root root 4096 May 3 13:25 eventlogs
drwxr-xr-x 2 root root 4096 May 3 14:00 ganglia
drwxr-xr-x 2 root root 4096 May 3 14:00 logs
-rw-r--r-- 1 root root 9434 May 3 14:01 sample_ADE_dataset.csv
/databricks/driver

In [0]:
ade_DF = spark.read\
                .option("header", "true")\
                .csv("/sample_ADE_dataset.csv")\
                .filter(F.col("label").isin(['True','False']))

ade_DF.show(truncate=50)

+--------------------------------------------------+-----+
 text|label|
+--------------------------------------------------+-----+
Do U know what Meds are R for bipolar depressio...|False|
# hypercholesterol: Because of elevated CKs (pe...| True|
Her weight, respirtory status and I/O should be...|False|
* DM - Pt had several episodes of hypoglycemia ...| True|
We report the case of a female acromegalic pati...| True|
2 . Calcipotriene 0.005% Cream Sig: One (1) App...|False|
Always tired, and possible blood clots. I was o...| True|
A difference in chemical structure between thes...|False|
10 . She was left on prednisone 20mg qd due to ...|False|
The authors suggest that risperidone may increa...| True|
- Per oral maxillofacial surgery there is no ev...|False|
@marionjross Cipro is just as bad! Stay away fr...|False|
A young woman with epilepsy had tonic-clonic se...| True|
Intravenous methotrexate is an effective adjunc...|False|
PURPOSE: To report new indocyanine green angiog...|False|
2 . Docusate Sodium 50 mg/5 mL Liquid [**Hospit...|False|
 consider neupogen.|False|
He was treated allopurinol and Rasburicase for ...|False|
Toxicity, pharmacokinetics, and in vitro hemodi...| True|
# thrombocytopenia: Secondary to chemotherapy a...| True|
+--------------------------------------------------+-----+
only showing top 20 rows

**With BioBert version of NER** (will be slower but more accurate)

In [0]:
import pyspark.sql.functions as F

ner_converter = NerConverter() \
  .setInputCols(["sentence", "token", "ner"]) \
  .setOutputCol("ner_chunk")\
  .setWhiteList(['ADE'])

ner_pipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    bert_embeddings,
    ade_ner_bert,
    ner_converter])


empty_data = spark.createDataFrame([[""]]).toDF("text")

ade_ner_model = ner_pipeline.fit(empty_data)

result = ade_ner_model.transform(ade_DF)

sample_df = result.select('text','ner_chunk.result')\
                  .toDF('text','ADE_phrases').filter(F.size('ADE_phrases')>0).toPandas()

In [0]:
import pandas as pd
pd.set_option('display.max_colwidth', 0)

In [0]:
sample_df.sample(20)

Out[22]:

,text,ADE_phrases
1,* DM - Pt had several episodes of hypoglycemia on lantus due to decreasing oral intake.,[hypoglycemia]
19,"Therefore, parenteral amiodarone was implicated as the cause of acute hepatitis in this patient.",[acute hepatitis]
12,"Effexor side effects; headaches, no appetite.","[Effexor side effects, headaches, no appetite]"
5,A young woman with epilepsy had tonic-clonic seizures during antineoplastic therapy with adriamycin and cisplatin.,[tonic-clonic seizures]
21,"2 years with no problems, then toe neuropathy for two years now and other foot problems because of this I assume. I stopped Lipitor after taking it for 2 years and toe neuropathy started. I also had stomach problems and pain. After stopping Lipitor, I thought my heart would jump out of my chest, but now stomach and heart are both OK - Feet are terrible thanks to Lipitor. I know many others that have had problems with Lipitor - feet and legs - and had to stop. All my blood test were normal. I weigh 114 and 5'2 . Hike, play golf and keep busy. It is a terrible drug and should be off the market.","[toe neuropathy, foot problems, toe neuropathy, stomach problems, pain]"
6,Intravenous methotrexate is an effective adjunct to steroid therapy in the treatment of steroid-resistant or life-threatening dermatomyositis-polyositis or dermatomyositis-polymyositis complicated by severe steroid-related effects.,"[life-threatening dermatomyositis-polyositis, dermatomyositis-polymyositis]"
14,"A transient tonic pupillary response, denervation supersensitivity, and abnormal visual-evoked potentials in quinine toxicity, to our knowledge, have not been previously reported.","[transient tonic pupillary response, denervation supersensitivity, abnormal visual-evoked potentials]"
22,Immunoglobulin G subclass 2 and 3 deficiencies were the only serologically detectable abnormalities.,"[Immunoglobulin G subclass 2 and 3 deficiencies, serologically detectable abnormalities]"
13,Hypokalemia after normal doses of neubulized albuterol (salbutamol).,[Hypokalemia]
0,# hypercholesterol: Because of elevated CKs (peaked at 819) the patient's Simvastatin was discontinued.,[elevated CKs]


**Doing the same with clinical embeddings version** (faster results)

In [0]:
import pyspark.sql.functions as F

ner_converter = NerConverter() \
  .setInputCols(["sentence", "token", "ner"]) \
  .setOutputCol("ner_chunk")\
  .setWhiteList(['ADE'])

ner_pipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    ade_ner,
    ner_converter])

empty_data = spark.createDataFrame([[""]]).toDF("text")

ade_ner_model = ner_pipeline.fit(empty_data)

result = ade_ner_model.transform(ade_DF)

result.select('text','ner_chunk.result')\
.toDF('text','ADE_phrases').filter(F.size('ADE_phrases')>0)\
.show(truncate=70)


+----------------------------------------------------------------------+----------------------------------------------------------------------+
 text| ADE_phrases|
+----------------------------------------------------------------------+----------------------------------------------------------------------+
# hypercholesterol: Because of elevated CKs (peaked at 819) the pat...| [elevated CKs]|
We report the case of a female acromegalic patient in whom multiple...| [multiple hepatic adenomas]|
Always tired, and possible blood clots. I was on Voltaren for about...| [blood clots that traveled to my eye, back pain]|
The authors suggest that risperidone may increase affect in patient...| [increase affect]|
A young woman with epilepsy had tonic-clonic seizures during antine...| [tonic-clonic seizures]|
Intravenous methotrexate is an effective adjunct to steroid therapy...| [dermatomyositis-polyositis]|
PURPOSE: To report new indocyanine green angiographic (ICGA) findin...| [indocyanine green angiographic (ICGA) findings]|
Toxicity, pharmacokinetics, and in vitro hemodialysis clearance of ...| [Toxicity]|
 # thrombocytopenia: Secondary to chemotherapy and MDS/AML concerns.| [thrombocytopenia]|
A fatal massive pulmonary embolus developed in a patient treated wi...| [fatal massive pulmonary embolus]|
# Maculopapular rash: over extremities, chest and back, thought [**...| [Maculopapular rash]|
 Hypokalemia after normal doses of neubulized albuterol (salbutamol).| [Hypokalemia]|
A transient tonic pupillary response, denervation supersensitivity,...|[transient tonic pupillary response, denervation supersensitivity, ...|
As per above, ID added Atovaquone for PCP [**Name9 (PRE) *] given t...| [BM suppression, liver damage]|
Electrocardiographic findings and laboratory data indicated a diagn...| [acute myocardial infarction]|
 Hepatic reactions to cyclofenil.| [Hepatic reactions]|
Therefore, parenteral amiodarone was implicated as the cause of acu...| [acute hepatitis]|
Vincristine levels were also assayed and showed a dramatic decline ...| [dramatic decline in postexchange levels]|
Eight days after the end of interferon treatment, he showed signs o...| [inability to sit]|
2 years with no problems, then toe neuropathy for two years now and...|[toe neuropathy, toe neuropathy, stomach problems, pain, heart woul...|
+----------------------------------------------------------------------+----------------------------------------------------------------------+
only showing top 20 rows

### creating sentence dataframe (one sentence per row) and getting ADE entities and categories

In [0]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

sentenceDetector = SentenceDetector()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")\
      .setExplodeSentences(True)

tokenizer = Tokenizer()\
      .setInputCols(["sentence"])\
      .setOutputCol("token")

bert_embeddings = BertEmbeddings.pretrained("biobert_pubmed_base_cased")\
      .setInputCols(["sentence", "token"])\
      .setOutputCol("embeddings")

embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["sentence", "embeddings"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")\
      .setStorageRef('biobert_pubmed_base_cased')

classsifierdl = ClassifierDLModel.pretrained("classifierdl_ade_biobert", "en", "clinical/models")\
      .setInputCols(["sentence", "sentence_embeddings"]) \
      .setOutputCol("class")\
      .setStorageRef('biobert_pubmed_base_cased')

ade_ner = MedicalNerModel.pretrained("ner_ade_biobert", "en", "clinical/models") \
      .setInputCols(["sentence", "token", "embeddings"]) \
      .setOutputCol("ner")
  
ner_converter = NerConverter() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ner_chunk")\
      .setWhiteList(['ADE'])

ner_clf_pipeline = Pipeline(
    stages=[documentAssembler, 
            sentenceDetector,
            tokenizer,
            bert_embeddings,
            embeddingsSentence,
            classsifierdl,
            ade_ner,
            ner_converter])

ade_Sentences = ner_clf_pipeline.fit(ade_DF)

biobert_pubmed_base_cased download started this may take some time.
Approximate size to download 386.4 MB
[ | ][OK!]
classifierdl_ade_biobert download started this may take some time.
Approximate size to download 21.8 MB
[ | ][OK!]
ner_ade_biobert download started this may take some time.
Approximate size to download 15.3 MB
[ | ][OK!]

In [0]:
import pyspark.sql.functions as F

ade_Sentences.transform(ade_DF).select('sentence.result','ner_chunk.result','class.result')\
.toDF('sentence','ADE_phrases','is_ADE').show(truncate=60)

+------------------------------------------------------------+---------------------------------------------+-------+
 sentence| ADE_phrases| is_ADE|
+------------------------------------------------------------+---------------------------------------------+-------+
 [Do U know what Meds are R for bipolar depression?]| []|[False]|
 [Currently #FDA approved #quetiapine AKA #Seroquel]| []|[False]|
[# hypercholesterol: Because of elevated CKs (peaked at 8...| [elevated CKs]|[False]|
[Her weight, respirtory status and I/O should be monitore...| []|[False]|
[* DM - Pt had several episodes of hypoglycemia on lantus...| [hypoglycemia]| [True]|
[We report the case of a female acromegalic patient in wh...| [hepatic adenomas]| [True]|
 [2 .]| []|[False]|
[Calcipotriene 0.005% Cream Sig: One (1) Appl Topical [**...| []|[False]|
 [Always tired, and possible blood clots.]| [tired, blood clots]|[False]|
[I was on Voltaren for about 4 years and all of the sudde...|[stroke, blood clots that traveled to my eye]|[False]|
[I had every test in the book done at the hospital, and t...| []|[False]|
 [I was completley healthy!]| [completley healthy]|[False]|
 [I am thinking it was from the voltaren.]| []|[False]|
[I have been off of the drug for 8 months now, and have n...| []|[False]|
[I started eating healthy and working out and that has he...| []|[False]|
 [I can now sleep all thru the night.]| []|[False]|
 [I wont take this again.]| []|[False]|
 [If I have the back pain, I will pop a tylonol instead.]| []|[False]|
[A difference in chemical structure between these two dru...| []|[False]|
 [10 .]| []|[False]|
+------------------------------------------------------------+---------------------------------------------+-------+
only showing top 20 rows

## Creating a pretrained pipeline with ADE NER, Assertion and Classifer

In [0]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
      .setInputCols(["sentence"])\
      .setOutputCol("token")

bert_embeddings = BertEmbeddings.pretrained("biobert_pubmed_base_cased")\
      .setInputCols(["sentence", "token"])\
      .setOutputCol("embeddings")

ade_ner = MedicalNerModel.pretrained("ner_ade_biobert", "en", "clinical/models") \
      .setInputCols(["sentence", "token", "embeddings"]) \
      .setOutputCol("ner")\
      .setStorageRef('biobert_pubmed_base_cased')

ner_converter = NerConverter() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ner_chunk")

assertion_ner_converter = NerConverter() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ass_ner_chunk")\
      .setWhiteList(['ADE'])

biobert_assertion = AssertionDLModel.pretrained("assertion_dl_biobert", "en", "clinical/models") \
      .setInputCols(["sentence", "ass_ner_chunk", "embeddings"]) \
      .setOutputCol("assertion")

embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["sentence", "embeddings"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")\
      .setStorageRef('biobert_pubmed_base_cased')

classsifierdl = ClassifierDLModel.pretrained("classifierdl_ade_conversational_biobert", "en", "clinical/models")\
      .setInputCols(["sentence", "sentence_embeddings"]) \
      .setOutputCol("class")

ade_clf_pipeline = Pipeline(
    stages=[documentAssembler, 
            tokenizer,
            bert_embeddings,
            ade_ner,
            ner_converter,
            assertion_ner_converter,
            biobert_assertion,
            embeddingsSentence,
            classsifierdl])

empty_data = spark.createDataFrame([[""]]).toDF("text")

ade_ner_clf_model = ade_clf_pipeline.fit(empty_data)

ade_ner_clf_pipeline = LightPipeline(ade_ner_clf_model)

biobert_pubmed_base_cased download started this may take some time.
Approximate size to download 386.4 MB
[ | ][OK!]
ner_ade_biobert download started this may take some time.
Approximate size to download 15.3 MB
[ | ][OK!]
assertion_dl_biobert download started this may take some time.
Approximate size to download 3 MB
[ | ][OK!]
classifierdl_ade_conversational_biobert download started this may take some time.
Approximate size to download 21.8 MB
[ | ][OK!]

In [0]:
classsifierdl.getStorageRef()

Out[32]: 'biobert_pubmed_base_cased'

In [0]:
text = 'Always tired, and possible blood clots. I was on Voltaren for about 4 years and all of the sudden had a minor stroke and had blood clots that traveled to my eye. I had every test in the book done at the hospital, and they couldnt find anything. I was completley healthy! I am thinking it was from the voltaren. I have been off of the drug for 8 months now, and have never felt better. I started eating healthy and working out and that has help alot. I can now sleep all thru the night. I wont take this again. If I have the back pain, I will pop a tylonol instead.'

light_result = ade_ner_clf_pipeline.fullAnnotate(text)

print (light_result[0]['class'][0].metadata)

chunks = []
entities = []
begin =[]
end = []

for n in light_result[0]['ner_chunk']:

    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 

import pandas as pd

df = pd.DataFrame({'chunks':chunks, 'entities':entities,
                    'begin': begin, 'end': end})

df

{'sentence': '0', 'False': '0.018514084', 'True': '0.9814859'}
Out[33]:

,chunks,entities,begin,end
0,tired,ADE,7,11
1,blood clots,ADE,27,37
2,Voltaren,DRUG,49,56
3,stroke,ADE,110,115
4,blood clots that traveled to my eye,ADE,125,159
5,voltaren,DRUG,301,308
6,tylonol,DRUG,548,554


In [0]:
import pandas as pd

text = 'I have always felt tired, but no blood clots. I was on Voltaren for about 4 years and all of the sudden had a minor stroke and had blood clots that traveled to my eye. I had every test in the book done at the hospital, and they couldnt find anything. I was completley healthy! I am thinking it was from the voltaren. I have been off of the drug for 8 months now, and have never felt better. I started eating healthy and working out and that has help alot. I can now sleep all thru the night. I wont take this again. If I have the back pain, I will pop a tylonol instead.'

print (text)

light_result = ade_ass_ner_model_lp_bert.fullAnnotate(text)[0]

chunks=[]
entities=[]
status=[]

for n,m in zip(light_result['ass_ner_chunk'],light_result['assertion']):
    
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    status.append(m.result)
        
df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status})

df

I have always felt tired, but no blood clots. I was on Voltaren for about 4 years and all of the sudden had a minor stroke and had blood clots that traveled to my eye. I had every test in the book done at the hospital, and they couldnt find anything. I was completley healthy! I am thinking it was from the voltaren. I have been off of the drug for 8 months now, and have never felt better. I started eating healthy and working out and that has help alot. I can now sleep all thru the night. I wont take this again. If I have the back pain, I will pop a tylonol instead.
Out[34]:

,chunks,entities,assertion
0,tired,ADE,present
1,blood clots,ADE,absent
2,stroke,ADE,present
3,blood clots that traveled to my eye,ADE,present
4,completley healthy,ADE,present


In [0]:
result = ade_ner_clf_pipeline.annotate('I just took an Advil 100 mg and it made me drowsy')

print (result['class'])
print(list(zip(result['token'],result['ner'])))

['False']
[('I', 'O'), ('just', 'O'), ('took', 'O'), ('an', 'O'), ('Advil', 'B-DRUG'), ('100', 'O'), ('mg', 'O'), ('and', 'O'), ('it', 'O'), ('made', 'O'), ('me', 'O'), ('drowsy', 'B-ADE')]

In [0]:
ade_ner_clf_model.write().overwrite().save('/databricks/driver/ade_pretrained_pipeline')

In [0]:
ade_ner_clf_model.stages

Out[44]: [DocumentAssembler_4b7411c32e88,
 REGEX_TOKENIZER_e4e8de870bb4,
 BERT_EMBEDDINGS_c6741c518b81,
 MedicalNerModel_4fc5b46ae2cf,
 NerConverter_888750c9fd7a,
 NerConverter_418666209360,
 ASSERTION_DL_2f4db8443148,
 SentenceEmbeddings_ee67930726a6,
 ClassifierDLModel_6edc7e323980]

In [0]:
from sparknlp.pretrained import PretrainedPipeline

ade_pipeline = PretrainedPipeline.from_disk('/databricks/driver/ade_pretrained_pipeline')

ade_pipeline.annotate('I just took an Advil 100 mg and it made me drowsy')

Out[43]: {'ner_chunk': ['Advil', 'drowsy'],
 'assertion': ['conditional'],
 'sentence_embeddings': ['I just took an Advil 100 mg and it made me drowsy'],
 'token': ['I',
 'just',
 'took',
 'an',
 'Advil',
 '100',
 'mg',
 'and',
 'it',
 'made',
 'me',
 'drowsy'],
 'ner': ['O', 'O', 'O', 'O', 'B-DRUG', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ADE'],
 'class': ['False'],
 'ass_ner_chunk': ['drowsy'],
 'embeddings': ['I',
 'just',
 'took',
 'an',
 'Advil',
 '100',
 'mg',
 'and',
 'it',
 'made',
 'me',
 'drowsy'],
 'sentence': ['I just took an Advil 100 mg and it made me drowsy']}

End of Notebook # 16